<a id='top'></a>
---

# Tri-Vibe Sensor Integration Notebook
* Language: Python 3.7  
* Required [Libraries](#libraries)
* Author: Leo Bach 12/18/2019
* Version: 5.01  
    (Object Oriented Design Implemented)  
    (CSV Storage Added)  
    (Charting Added)  
    (HTML Storage Added)  
* Email: leo@machinesaver.net  
* Phone: +1(832)581-9908

### Table of Contents:
[Part 1: Required Libraries for TRIVIBE CLASS](#libraries)  
[Part 2: TRIVIBE CLASS](#trivibeclass)  
[Part 3: Required Libraries for NOTEBOOK](#notebooklibraries)  
[Part 4: Find Computer's RS485 Port](#checkport)  
[Part 5: Available TRIVIBE Methods (Functions)](#methodlist)  
[Part 6: Create a Sensor Object from TRIVIBE CLASS](#instantiation)  
[Part 7: Available TRIVIBE Attributes (Properties)](#attributelist)  
[Part 8: Example Static Attributes - Sensor Info](#staticattributes)  
[Part 9: Example Uptime](#exampleuptime)  
[Part 10: Example Temperature](#exampletemperature)  
[Part 11: Example Vibration](#examplevibration)  
[Part 12: Example Easy Snapshot](#easysnapshot)  
[Part 13: Example Snapshot Storage (CSV)](#storecsv)  
[Part 14: Example Snapshot Chart (Plotly)](#chartcsv)  
[Part 15: Example Snapshot Chart Storage (HTML)](#storehtml)  
[Part 16: Example Advanced Snapshot](#advancedsnapshot)  

<a id='libraries'></a>
---

# Part 1: Required Libraries for [TRIVIBE CLASS](#trivibeclass)  

In [1]:
# import required libraries for TRIVIBE CLASS
import minimalmodbus                        # modbus master library which supports Modbus RTU & Modbus ASCII but not Modbus TCP
import time                                 # use sleep to wait while sensor processes are happening + used to take timestamp when data is collected
from datetime import datetime               # manipulate timestamp to human readable date
import numpy as np                          # number manipulation - calculate snapshot reads, create time arrays (twf - x-axis) and frequency bins arrays (spectrum - x-axis)
from scipy.fftpack import fft               # fast fourier transform algorithm
from pathlib import Path
from openpyxl import Workbook
import plotly.graph_objects as go           # charting library
import xlsxwriter

# remove libraries?
import os                                   # used to save snapshots to /snapshots directory within Machine Saver's Jupyter notebook
import matplotlib.pyplot as plt             # charting library
import pandas as pd                         # used to manipulate numbers
import os                                   # used to save snapshots to /snapshots directory within Machine Saver's Jupyter notebook
import math                                 # used to manipulate numbers

<a id='trivibeclass'></a>
---

# Part 2: TRIVIBE_IMPACT CLASS

In [2]:
class TRIVIBE_IMPACT:
    manufacturer = 'Machine Saver Inc.'
    
    def __init__(self, slaveAddress, serialPort):
        self.serialPort = serialPort
        self.modbus = minimalmodbus.Instrument(self.serialPort, slaveAddress)
        self._setSerialPortSettings()
        
        self.info = {
            "serial": self._getSerialNumber(),
            "revision": self._getRevision(),
            "board_type": self._getBoardType(),
            "mode": self._getMode(),
            "internal_accelerometer_1":{},
            "internal_accelerometer_2":{}
        }
        self._getSensitivity()
        
        self.uptime={}
        self.temperature={}
        self.impact={}
        self.alarms={}
        self.snapshot={}
        
        self.setImpactDefaults()

### Initialization and Configuration Functions
    def _setSerialPortSettings(self):
        '''Update serial communication settings to talk to Tri-Vibe'''
        self.modbus.serial.baudrate = 115200           # Baudrate fixed 115200
        self.modbus.serial.bytesize = 8                # Bytesize fixed 8
        self.modbus.serial.parity   = "N"              # Parity fixed None 
        self.modbus.serial.stopbits = 1                # Stopbits fixed 1
        self.modbus.serial.timeout  = 0.25             # Seconds
        self.modbus.close_port_after_each_call = True  # Helps communication for Windows Devices (can be set to False on many Linux devices)
        self.modbus.mode = minimalmodbus.MODE_RTU      # modbus mode fixed RTU Mode
        self.modbus.clear_buffers_before_each_transaction = True
        time.sleep(0.1)
    
    def _getRevision(self):
        '''Reads Modbus Register 0. Scales return value to Revision. Updates Trivibe Object Property info["revision"].'''
        sensor_revision = self.modbus.read_register(0, functioncode=3)  # Revision at time of writing 112
        return(sensor_revision-768)                                  # Remove the 768 from the output of register 0 to get the sensor's revision

    def _getSerialNumber(self):
        '''Reads Modbus Register 26. Updates Trivibe Object Property info["serial"].'''
        serial = self.modbus.read_long(26, functioncode=3)
        return serial

    def _getMode(self):
        '''Reads Modbus Register 14.'''
        mode = self.modbus.read_register(14, functioncode=3)
        return(self._modbus_map_mode(mode))

    def _modbus_map_mode(self, x):
        return {
            'vibration':1,
            1:'vibration',
            'impact':2,
            2:'impact',
            'detonation':4,
            4:'detonation',
            'extended_impact':8,
            8:'extended_impact',
        }.get(x, 'error')    # 'error' is default if x not found
    
    def _modbus_map_alarm_status(self, x):
        return {
            0:'none',
            1:'low',
            2:'high',
            3:'highhigh',
            4:'normal',
        }.get(x, 'error')    # 'error' is default if x not found

    def _getBoardType(self):
        '''Reads Modbus Register 366. Updates Trivibe Object Property info[board_type].'''
        return(self._modbus_map_board_type(self.modbus.read_register(366, functioncode=3)))

    def _modbus_map_board_type(self, x): # checks the board type
        return {
            0:['Unset','Unset'],
            1:['2g','2g'],
            2:['2g','4g'],
            3:['4g','2g'],
            4:['2g','25g'],
            5:['4g','25g'],
            6:['25g','2g'],
            7:['25g','4g'],
            8:['4g','10g'],
            9:['10g','4g'],
            10:['100g','10g'],
        }.get(x, 'error')    # 'error' is default if x not found

    def _frequency_map(self, x): # checks the board type
        return {
            '2g':1000,
            '4g':1000,
            '10g':5000,
            '20g':5000,
            '25g':5000,
            '100g':5000,
        }.get(x, 'error')    # 'error' is default if x not found

    def _amplitude_map(self, x): # checks the board type
        return {
            '2g':2,
            '4g':4,
            '10g':10,
            '20g':20,
            '25g':25,
            '100g':100,
        }.get(x, 'error')    # 'error' is default if x not found

    def _getSensitivity(self):
        '''Reads Modbus Registers 299-310 . Updates Trivibe Object Property
        info.internal_accelerometer_1["max_amplitude_gPK"]
                                    ,["max_frequency_hz"]
                                    ,["axis_1_sensitivity"]
                                    ,["axis_2_sensitivity"]
                                    ,["axis_3_sensitivity"]
        info.internal_accelerometer_2["max_amplitude_gPK"]
                                    ,["max_frequency_hz"]
                                    ,["axis_1_sensitivity"]
                                    ,["axis_2_sensitivity"]
                                    ,["axis_3_sensitivity"]'''
        s1_a1_sensitivity = self.modbus.read_float(299, functioncode=3, number_of_registers=2, byteorder=0)
        s1_a2_sensitivity = self.modbus.read_float(301, functioncode=3, number_of_registers=2, byteorder=0)
        s1_a3_sensitivity = self.modbus.read_float(303, functioncode=3, number_of_registers=2, byteorder=0)
        s2_a1_sensitivity = self.modbus.read_float(305, functioncode=3, number_of_registers=2, byteorder=0)
        s2_a2_sensitivity = self.modbus.read_float(307, functioncode=3, number_of_registers=2, byteorder=0)
        s2_a3_sensitivity = self.modbus.read_float(309, functioncode=3, number_of_registers=2, byteorder=0)
        self.info.setdefault("internal_accelerometer_1",{})
        self.info.get("internal_accelerometer_1").setdefault("max_amplitude_gPK", self._amplitude_map(self.info.get("board_type")[0]))
        self.info.get("internal_accelerometer_1").setdefault("max_frequency_hz", self._frequency_map(self.info.get("board_type")[0]))
        self.info.get("internal_accelerometer_1").setdefault("axis_1_sensitivity",s1_a1_sensitivity)
        self.info.get("internal_accelerometer_1").setdefault("axis_2_sensitivity",s1_a2_sensitivity)
        self.info.get("internal_accelerometer_1").setdefault("axis_3_sensitivity",s1_a3_sensitivity)
        self.info.setdefault("internal_accelerometer_2",{})
        self.info.get("internal_accelerometer_2").setdefault("max_amplitude_gPK", self._amplitude_map(self.info.get("board_type")[1]))
        self.info.get("internal_accelerometer_2").setdefault("max_frequency_hz", self._frequency_map(self.info.get("board_type")[1]))
        self.info.get("internal_accelerometer_2").setdefault("axis_1_sensitivity",s2_a1_sensitivity)
        self.info.get("internal_accelerometer_2").setdefault("axis_2_sensitivity",s2_a2_sensitivity)
        self.info.get("internal_accelerometer_2").setdefault("axis_3_sensitivity",s2_a3_sensitivity)

    def setImpactDefaults(self):
        self._unlockControlReg()
        self._setMode('impact')
        self.updateMachineSpeed()
        self.updateAlertThreshold()
        self.updateDangerThreshold()
        self._lockControlReg()
        self.reboot()
        self._unlockAlarmReg()
        self._impactAlarmDefaults()
        self._lockAlarmReg()
        self._alarmStatus()
        
    def _impactAlarmDefaults(self):
        print("Trip Delay")
        trip_delay = 0
        self.modbus.write_register(311, trip_delay)
        print(self.modbus.read_register(311))
        
        print("Channel 1")        
        values = [6, 5]
        self.modbus.write_registers(312, values)
        print(self.modbus.read_register(312))
        print(self.modbus.read_register(313))
        self.modbus.write_float(314, -100000)
        print(self.modbus.read_float(314))
        self.modbus.write_float(316, 1)
        print(self.modbus.read_float(316))
        self.modbus.write_float(318, 100000)
        print(self.modbus.read_float(318))
        self.modbus.write_float(320, 0)
        print(self.modbus.read_float(320))
        
        print("Channel 2")
        values = [6, 5]
        self.modbus.write_registers(322, values)
        print(self.modbus.read_register(322))
        print(self.modbus.read_register(323))
        self.modbus.write_float(324, -100000)
        print(self.modbus.read_float(324))
        self.modbus.write_float(326, 1)
        print(self.modbus.read_float(326))
        self.modbus.write_float(328, 100000)
        print(self.modbus.read_float(328))
        self.modbus.write_float(330, 0)
        print(self.modbus.read_float(330))

        print("Channel 3")
        values = [6, 5]
        self.modbus.write_registers(332, values)
        print(self.modbus.read_register(332))
        print(self.modbus.read_register(333))
        self.modbus.write_float(334, -100000)
        print(self.modbus.read_float(334))
        self.modbus.write_float(336, 1)
        print(self.modbus.read_float(336))
        self.modbus.write_float(338, 100000)
        print(self.modbus.read_float(338))
        self.modbus.write_float(340, 0)
        print(self.modbus.read_float(340))
        
        print("Channel 4")
        values = [6, 5]
        self.modbus.write_registers(342, values)
        print(self.modbus.read_register(342))
        print(self.modbus.read_register(343))
        self.modbus.write_float(344, -100000)
        print(self.modbus.read_float(344))
        self.modbus.write_float(346, 1)
        print(self.modbus.read_float(346))
        self.modbus.write_float(348, 100000)
        print(self.modbus.read_float(348))
        self.modbus.write_float(350, 0)
        print(self.modbus.read_float(350))
        
    def updateMachineSpeed(self, rpm=960):
        self.modbus.write_register(372, rpm)
        speed = self.modbus.read_register(372)
        try:
            self.impact.update({"machine_speed_rpm":speed})
        except KeyError:
            self.impact.setdefault({"machine_speed_rpm":speed})

    def updateAlertThreshold(self, gpk=5):
        self.modbus.write_float(368, gpk)
        gpk = self.modbus.read_float(368)
        try:
            self.impact.update({"alert_threshold_gpk":gpk})
        except KeyError:
            self.impact.setdefault({"alert_threshold_gpk":gpk})

    def updateDangerThreshold(self, gpk=15):
        self.modbus.write_float(370, gpk)
        gpk = self.modbus.read_float(370)
        try:
            self.impact.update({"danger_threshold_gpk":gpk})
        except KeyError:
            self.impact.setdefault({"danger_threshold_gpk":gpk})

#     def updateImpact(self):
#         impact_value = self.modbus.read_registers(45, 3, functioncode=3)
#         try:
#             self.impact.update({"impact_counts":impact_value[0],
#                                 "ha_impact_counts":impact_value[1],
#                                 "highest_amp_gpk":impact_value[2]*0.100})
#         except KeyError:
#             self.impact.setdefault({"impact_counts":impact_value[0],
#                                 "ha_impact_counts":impact_value[1],
#                                 "highest_amp_gpk":impact_value[2]*0.100})

    def updateImpact(self):
        severity = self.modbus.read_float(234, functioncode=3, number_of_registers=2, byteorder=0)
        alert_counts = self.modbus.read_float(230, functioncode=3, number_of_registers=2, byteorder=0)
        danger_counts = self.modbus.read_float(232, functioncode=3, number_of_registers=2, byteorder=0)
        temperature = self.modbus.read_float(236, functioncode=3, number_of_registers=2, byteorder=0)
        try:
            self.impact.update({"alert_counts":alert_counts,
                                "danger_counts":danger_counts,
                                "severity":severity,
                                "temperature":temperature})
        except KeyError:
            self.impact.setdefault({"alert_counts":alert_counts,
                                    "danger_counts":danger_counts,
                                    "severity":severity,
                                    "temperature":temperature})
        self._clearHighestValues()
        time.sleep(0.2)

    def _clearHighestValues(self):
        CLEAR_CHANNEL_1 = 1
        CLEAR_CHANNEL_2 = 2
        CLEAR_CHANNEL_3 = 4
        CLEAR_CHANNEL_4 = 8
        CLEAR_CHANNEL_ALL = 15
        ALARM_CONTROL_REGISTER = 238
        self.modbus.write_register(ALARM_CONTROL_REGISTER, CLEAR_CHANNEL_ALL)

### Modbus Functions
    def reboot(self):
        self.modbus.write_register(1, 1)
        time.sleep(6)

    def _unlockControlReg(self):
        UNLOCK=45555
        self.modbus.write_register(1, UNLOCK)

    def _lockControlReg(self):
        LOCK=45556
        self.modbus.write_register(1, LOCK)

    def _unlockAlarmReg(self):
        UNLOCK=42074
        self.modbus.write_register(1, UNLOCK)

    def _lockAlarmReg(self):
        LOCK=42075
        self.modbus.write_register(1, LOCK)

    def _setMode(self, mode='vibration'):
        mode = self._modbus_map_mode(mode)
        self.modbus.write_register(373, mode)
    
    def _alarmStatus(self):
        status = self.modbus.read_registers(226, 4)
        status_text = []
        for each in status:
            status_text.append(self._modbus_map_alarm_status(each))
        return status_text
    
    def _alarmSettings(self):
        status = self.modbus.read_registers(312, 40)
        status_text = []
        for each in status:
            status_text.append(each)
        return status_text

    def _alarmHighest(self):
        status = self.modbus.read_registers(230, 8)
        status_text = []
        for each in status:
            status_text.append(each)
        return status_text

    def changeSlaveAddress(self, newSlaveAddress):
        if(newSlaveAddress < 2 or newSlaveAddress > 254):
            raise ValueError('Invalid Slave Address: must be an integer >1 and <254')
        else:
            self._unlockControlReg()
            self.modbus.write_register(367, newSlaveAddress)         # Change Sensor Slave Address
            self._lockControlReg()
            self.reboot()
            self.modbus = minimalmodbus.Instrument(self.port, newSlaveAddress)    # Update Slave Address to New Address for our Tri-Vibe Object in minimal modbus

### OVERALL METHODS
    def updateTemperature(self):
        '''Reads Modbus Register 31. Scales to Celcius and Fahrenheit. Updates Trivibe Object Properties [temperature_C, temperature_F].'''
        temperature = self.modbus.read_register(31, functioncode=3)
        self.temperature = {
            "celcius":(temperature/10),
            "fahrenheit":((temperature/10*9/5) + 32)
        }
        try:
            self.temperature.update({
            "celcius":(temperature/10),
            "fahrenheit":((temperature/10*9/5) + 32)
        })
        except KeyError:
            self.temperature.setdefault({
            "celcius":(temperature/10),
            "fahrenheit":((temperature/10*9/5) + 32)
        })
        time.sleep(0.2)
        
    def updateUptime(self):
        '''Returns an Array with the [minutes,hours,days] a sensor has been running. If powered off or rebooted these values reset.'''
        sensor_uptime = self.modbus.read_registers(5, 3, functioncode=3)  # register_number, number_of_registers, function_code
        try:
            self.uptime.update({
            "minutes":sensor_uptime[0],
            "hours":sensor_uptime[1],
            "days":sensor_uptime[2]
        })
        except KeyError:
            self.uptime.setdefault({
            "minutes":sensor_uptime[0],
            "hours":sensor_uptime[1],
            "days":sensor_uptime[2]
        })
        time.sleep(0.1)

### SNAPSHOT METHODS
    def setSnapshotTime(self, milliseconds=1000):
        '''Set the length of time for the sensor to capture. Validates input: is 10ms increments between 10 & 30000.'''
        if(milliseconds % 10 == 0 and 10<milliseconds<30000):
            self.modbus.write_register(35, milliseconds, number_of_decimals=0, functioncode=16, signed=False)
            capture_time = self.modbus.read_register(35, functioncode=3)
            self.snapshot.setdefault("capture_time_ms", capture_time)
            self.snapshot.setdefault("capture_time_s", capture_time/1000)
        else:
            raise ValueError('milliseconds should be a 10ms increment between 10 and 30000.')
     
    def setSnapshotSamples(self, samples=16384):
        '''Set the how many samples per axis for the sensor to capture over the capture time. Valid inputs powers of 2 from 256 - 1048576.'''
        self.modbus.write_long(36, samples, signed=False, byteorder=0)
        samples_per_axis = self.modbus.read_long(36, functioncode=3, signed=False, byteorder=0)
        self.snapshot.setdefault("samples_per_axis", samples_per_axis)

    def _modbus_map_internal_accel(self, x):
        '''Accepts a value of 5 or 6 and returns a string for the internal accelerometer that was selected for snapshot collection.'''
        return {
            5:self.info["internal_accelerometer_1"],
            6:self.info["internal_accelerometer_2"],
        }.get(x, 'error')    # 'error' is default if x not found

    def setSnapshotAccel(self, internalAccel=5):
        '''Sets which of the two accelerometers inside the Tri-Vibe sensor to target when a snapshot is triggered. Valid Inputs: 5 or 6.'''
        self.modbus.write_register(32, internalAccel, number_of_decimals=0, functioncode=16, signed=False)
        selected_internal_accelerometer = self.modbus.read_register(32, functioncode=3)
        selected = self._modbus_map_internal_accel(selected_internal_accelerometer)
        try:
            self.snapshot.update({"internal_accelerometer":selected})
        except KeyError:
            self.snapshot.setdefault("internal_accelerometer", selected)

    def _modbus_map_capture_engine_status(self, x):
        return {
            0:"Idle",
            1:"Complete",
            2:"In Progress",
            3:"Start Error",
            4:"Timeout Error",
        }.get(x, 'error')    # 'error' is default if x not found

    def updateSnapshotEngineStatus(self):
        capture_engine_status = self.modbus.read_register(34, functioncode=3)
        time.sleep(0.1)
        try:
            self.snapshot.update({"capture_engine_status":self._modbus_map_capture_engine_status(capture_engine_status)})
        except KeyError:
            self.snapshot.setdefault("capture_engine_status", self._modbus_map_capture_engine_status(capture_engine_status))
    
    def triggerSnapshot(self):
        '''Trigger the sensor to begin collecting and storing triaxial RAW accelerometer data in sensor memory. Writes a value of 1 to register 33.'''
        self.modbus.write_register(33, 1, functioncode=16)
        try:
            self.snapshot.update({"timestamp":int(str(time.time())[slice(10)])})
        except KeyError:
            self.snapshot.setdefault("timestamp", int(str(time.time())[slice(10)]))
        try:
            self.snapshot.update({"datetime":str(datetime.fromtimestamp(self.snapshot.get("timestamp")))})
        except KeyError:
            self.snapshot.setdefault("datetime", str(datetime.fromtimestamp(self.snapshot.get("timestamp"))))
        try:
            self.snapshot.update({"triaxial_RAW_samples":[]})
        except KeyError:
            self.snapshot.setdefault("triaxial_RAW_samples",[])
        try:
            self.snapshot.update({"current_chunk":0})
        except KeyError:
            self.snapshot.setdefault("current_chunk",0)
        self._calcSnapshotTotalSamples()
        self._calcSnapshotChunks()
        self._createTimeArray()

    def _calcSnapshotTotalSamples(self):
        '''Gets the total number of samples held on the tri-vibe for the triaxial_raw data set.'''
        total_samples = self.snapshot.get("samples_per_axis")*3
        try:
            self.snapshot.update({"total_samples":total_samples})
        except KeyError:
            self.snapshot.setdefault("total_samples", total_samples)

    def _calcSnapshotChunks(self):
        '''The tri-vibe can return sets of 122 samples of the triaxial_raw  data set per read. This function calculates how many times we must read modbus registers 50 - 123 to get the complete data set.'''
        try:
            self.snapshot.update({"chunks_to_read":int(np.ceil(self.snapshot.get("total_samples")/122))})
        except KeyError:
            self.snapshot.setdefault("chunks_to_read", int(np.ceil(self.snapshot.get("total_samples")/122)))

    def _createTimeArray(self):
        '''Creates the x-axis of the timewave form chart.'''
        time_ms = self.snapshot.get("capture_time_ms")
        samples_per_axis = self.snapshot.get("samples_per_axis")
        step = time_ms/samples_per_axis
        if(step < 0.02):
            raise ValueError("Sample_Rate exceeds Tri-Vibe built in ADC capability of 50,000 samples per second. Please change capture_time or samples_per_axis so that time/samples >= 0.02")
        else:
            start = 0
            stop = time_ms
            time_array = np.arange(start, stop, step)
            try:
                self.snapshot.update({"time_array_ms":time_array})
            except KeyError:
                self.snapshot.setdefault("time_array_ms",time_array)

    def collectSnapshot(self):
        while(self.snapshot.get("current_chunk")<self.snapshot.get("chunks_to_read")):
                self.collectSnapshotChunk()
                self.snapshot["current_chunk"]+=1
        self._sliceToAxis()

    def collectSnapshotChunk(self):
        chunk_read = self.modbus.read_registers(49, 123, functioncode=3)
        try:
            self.snapshot.update({"start_sample_number_last_chunk_read":chunk_read[0]})
        except KeyError:
            self.snapshot.setdefault("start_sample_number_last_chunk_read", chunk_read[0])
        chunk_read.pop(0)
        for each in chunk_read:
            self.snapshot["triaxial_RAW_samples"].append(each)
        
    def createSnapshotXLSX(self):
        cwd = os.getcwd()                                # save Jupyter notebook's directory
        os.chdir('snapshots')                            # move to snapshots directory
        # Excel Sheet Storage of Data
        # Create a workbook and add a worksheet.
        workbook = xlsxwriter.Workbook(str(self.info.get("serial"))+'_'+str(self.snapshot.get("timestamp"))+'.xlsx')
        worksheet = workbook.add_worksheet()

        # Start from the first cell. Rows and columns are zero indexed.

        # Add Column Headers
        worksheet.write(0, 0, "timestamp")
        worksheet.write(0, 1, "time_ms")
        worksheet.write(0, 2, "a1_twf")
        worksheet.write(0, 3, "a2_twf")
        worksheet.write(0, 4, "a3_twf")
        worksheet.write(0, 5, "a1_sensitivity")
        worksheet.write(0, 6, "a2_sensitivity")
        worksheet.write(0, 7, "a3_sensitivity")
        worksheet.write(0, 8, "capture_time_ms")
        worksheet.write(0, 9, "samples_per_axis")

        row = 1
        col = 0
        # Iterate over timestamp data and write it out row by row.
        for each in self.snapshot.get("time_array_ms"):
            worksheet.write(row, col, self.snapshot.get("timestamp"))
            row += 1

        # Iterate over Amplitude data and write it out row by row.
        row = 1
        col = 1
        for each in self.snapshot.get("time_array_ms"):
            worksheet.write(row, col, each)
            row += 1

        row = 1
        col = 2
        for each in self.snapshot.get("axis_1_RAW"):
            worksheet.write(row, col, each)
            row += 1

        row = 1
        col = 3
        for each in self.snapshot.get("axis_2_RAW"):
            worksheet.write(row, col, each)
            row += 1

        row = 1
        col = 4
        for each in self.snapshot.get("axis_3_RAW"):
            worksheet.write(row, col, each)
            row += 1

        row = 1
        col = 5
        worksheet.write(row, col, self.snapshot.get("internal_accelerometer").get("axis_1_sensitivity"))

        row = 1
        col = 6
        worksheet.write(row, col, self.snapshot.get("internal_accelerometer").get("axis_2_sensitivity"))

        row = 1
        col = 7
        worksheet.write(row, col, self.snapshot.get("internal_accelerometer").get("axis_1_sensitivity"))

        row = 1
        col = 8
        worksheet.write(row, col, self.snapshot.get("capture_time_ms"))

        row = 1
        col = 9
        worksheet.write(row, col, self.snapshot.get("samples_per_axis"))

        workbook.close()
        os.chdir(cwd)
        print('Saved '+str(self.info.get("serial"))+'_'+str(self.snapshot.get("timestamp"))+'.xlsx''.xlsx File to /snapshots Directory')


    def _createFreqArray(self):
        n = np.size(self.snapshot.get("time_array")) #5000
        print(n)
        sample_rate = self.snapshot.get("sample_rate")
        print(sample_rate)
        freq_array = (sample_rate/2)*np.linspace(0,1,n//2)
        trimmed_frequency_array = freq_array
        self.snapshot.setdefault("freq_array",trimmed_frequency_array)

    def processCollectedSnapshot(self):
        if "axis_1_acceleration" not in self.snapshot:
            self.snapshot["axis_1_acceleration"]=[]
            self.snapshot["axis_2_acceleration"]=[]
            self.snapshot["axis_3_acceleration"]=[]
            self.processCollectedSnapshot()
        else:
            self._sliceToAxis()
            self.snapshot["axis_1_acceleration"] = self._processAxisAcceleration(self.snapshot["axis_1_RAW"],self.snapshot["internal_accelerometer"]["axis_1_sensitivity"])
            self.snapshot["axis_2_acceleration"] = self._processAxisAcceleration(self.snapshot["axis_2_RAW"],self.snapshot["internal_accelerometer"]["axis_2_sensitivity"])
            self.snapshot["axis_3_acceleration"] = self._processAxisAcceleration(self.snapshot["axis_3_RAW"],self.snapshot["internal_accelerometer"]["axis_3_sensitivity"])

    def _sliceToAxis(self):
        '''Takes the triaxial RAW Snapshot array and splits it into individual arrays of the individual axes. Note: We read 122 blocks of data at a time via Modbus to get the RAW samples... when sliced based on samples_per_axis the additional zeros sent in the last chunk read are ignored as expected.'''
        try:
            self.snapshot.update({"axis_1_RAW":self.snapshot["triaxial_RAW_samples"][0:self.snapshot["samples_per_axis"]],
                                  "axis_2_RAW":self.snapshot["triaxial_RAW_samples"][self.snapshot["samples_per_axis"]:(self.snapshot["samples_per_axis"]*2)],
                                  "axis_3_RAW":self.snapshot["triaxial_RAW_samples"][(self.snapshot["samples_per_axis"]*2):(self.snapshot["samples_per_axis"]*3)]})
        except KeyError:
            self.snapshot.setDefault({"axis_1_RAW":self.snapshot["triaxial_RAW_samples"][0:self.snapshot["samples_per_axis"]],
                                  "axis_2_RAW":self.snapshot["triaxial_RAW_samples"][self.snapshot["samples_per_axis"]:(self.snapshot["samples_per_axis"]*2)],
                                  "axis_3_RAW":self.snapshot["triaxial_RAW_samples"][(self.snapshot["samples_per_axis"]*2):(self.snapshot["samples_per_axis"]*3)]})

    
    def _processAxisAcceleration(self, array, sensitivity):
        '''Takes an axis array of RAW accelerometer data and converts to the Timewave Form values in Acceleration.'''
        new_array=[]
        constant_k = 3000/65535
        virtual_zero = self._getVirtualZero(min(array),max(array))
        for index in range(len(array)):
            if array[index] > virtual_zero:
                new_array.append((abs(array[index]-virtual_zero)*constant_k)/sensitivity)
            elif array[index] < virtual_zero:
                new_array.append((-abs(array[index]-virtual_zero)*constant_k)/sensitivity)
            else:
                new_array.append(0)
        return(new_array)
            
            
    def _getVirtualZero(self, minimum, maximum):
        '''Returns the zero position for an axis array.'''
        return((maximum-minimum)/2+minimum)
    
    def easySnapshot(self):
        self.setSnapshotTime()
        self.setSnapshotSamples()
        self.setSnapshotAccel()
        while(self.snapshot["capture_engine_status"]!="Idle"):
            self.updateSnapshotEngineStatus()
            time.sleep(0.5)
        self.triggerSnapshot()
        while(self.snapshot["capture_engine_status"]!="Complete"):
            self.updateSnapshotEngineStatus()
            time.sleep(0.5)
        self.collectSnapshot()
        self.processCollectedSnapshot()
    
        

<a id='notebooklibraries'></a>
---

# Part 3: Required Libraries for NOTEBOOK

In [3]:
# notebook libraries
import serial.tools.list_ports              # used to find COM Port of your USB to RS485 Converter

<a id='checkport'></a>
---

# Part 4: Find Computer's RS485 Port
#### Windows Users Port Address Should Look Like: 'COM3'
#### Linux Users Port Address Should Look Like: '/dev/ttyUSB0'
#### 1. You should pass the RS485 Port Address as the 2nd Argument when creating a sensor object.

In [4]:
def getPorts():
    ports = list(serial.tools.list_ports.comports())
    if len(ports)==0:
        raise Exception('You must plug in an RS485 to USB Converter or have an RS485 Port on your computer to run the rest of this notebook.')
    else:
        for p in ports:
            return(p)

print(getPorts())

COM3 - USB-SERIAL CH340 (COM3)


<a id='methodlist'></a>
---

# Part 5: Available TRIVIBE_IMPACT Methods (Functions)  
#### 1. Methods do one of the following things:  
* gather information from the sensor and store them in attributes.  
* change information already stored in attributes.  
* take information stored in attributes and store on the local filesystem  
* take information stored on the local filesystem and visualize it  

#### 2. The list of methods below are the methods built for users of of the TRIVIBE CLASS
#### 3. Implementation details can be read in the [TRIVIBE_IMPACT CLASS](#trivibeclass)  
#### 4. Each of the Methods will be shows in the [examples section](#examples)

In [5]:
method_list = [func for func in dir(TRIVIBE_IMPACT) if callable(getattr(TRIVIBE_IMPACT, func))and not func.startswith("__") and not func.startswith("_")]
for each in method_list:
    print(each+'()')

changeSlaveAddress()
collectSnapshot()
collectSnapshotChunk()
createSnapshotXLSX()
easySnapshot()
processCollectedSnapshot()
reboot()
setImpactDefaults()
setSnapshotAccel()
setSnapshotSamples()
setSnapshotTime()
triggerSnapshot()
updateAlertThreshold()
updateDangerThreshold()
updateImpact()
updateMachineSpeed()
updateSnapshotEngineStatus()
updateTemperature()
updateUptime()


<a id='instantiation'></a>
---

# Part 6: Create a Sensor Object from [TRIVIBE_IMPACT CLASS](#trivibeclass)
#### 1. Required Arguments: MODBUS SLAVE ADDRESS and RS485 PORT ADDRESS.
#### 2. To reuse the Sensor Object throughout the project (NOTEBOOK), It must be assigned a variable name.
#### 3. Machine Saver prefers to use the Machine Name and Sensor's Position on the Machine as the variable name.  
#### 4. For examples taken from your own connected Tri-Vibe Sensor, simply edit the arguments below with your MODBUS SLAVE ADDRESS and RS485 PORT ADDRESS. Then re-run this notebook.

# TODO: Add Motor/Pump Picture with Sensors

In [6]:
motor_inboard = TRIVIBE_IMPACT(96, 'COM3')

Trip Delay
0
Channel 1
6
5
-100000.0
1.0
100000.0
0.0
Channel 2
6
5
-100000.0
1.0
100000.0
0.0
Channel 3
6
5
-100000.0
1.0
100000.0
0.0
Channel 4
6
5
-100000.0
1.0
100000.0
0.0


<a id='attributelist'></a>
---

# Part 7: Available TRIVIBE_IMPACT Attributes (Properties)
#### 1. Attributes Store Information about a Tri-Vibe Sensor and its relevant data.

In [7]:
attribute_list = motor_inboard.__dict__
for each in attribute_list:
    print(each)

serialPort
modbus
info
uptime
temperature
impact
alarms
snapshot


<a id='staticattributes'></a>
---

# Part 8: Static Attributes
#### 1. Some Attributes should not change throughout the lifecycle of the Tri-Vibe Sensor.
#### 2. The [Attributes](#attributelist) below are created when the Sensor Object is [created](#instantiation) and are not intended to change.

In [8]:
motor_inboard.info

{'serial': 41996,
 'revision': 112,
 'board_type': ['25g', '2g'],
 'mode': 'impact',
 'internal_accelerometer_1': {'max_amplitude_gPK': 25,
  'max_frequency_hz': 5000,
  'axis_1_sensitivity': 56.11345291137695,
  'axis_2_sensitivity': 56.01784133911133,
  'axis_3_sensitivity': 35.191673278808594},
 'internal_accelerometer_2': {'max_amplitude_gPK': 2,
  'max_frequency_hz': 1000,
  'axis_1_sensitivity': 656.8261108398438,
  'axis_2_sensitivity': 655.9004516601562,
  'axis_3_sensitivity': 649.1815185546875}}

<a id='exampleuptime'></a>
---

# Part 9: Example Uptime
#### 1. Let's call the uptime [Attribute](#attributelist) for our example sensor.

In [9]:
motor_inboard.uptime

{}

#### 2. Notice how the uptime is an empty object within the sensor object
#### 3. This is because we have not yet called the Method which updates this Attribute
#### 4. Lets do that now using the updateUptime() [Method](#methodlist)

In [10]:
motor_inboard.updateUptime()

#### 5. Let's check the uptime [Attribute](#attributelist) after running the updateUptime() [Method](#methodlist)

In [11]:
motor_inboard.uptime

{'minutes': 0, 'hours': 0, 'days': 0}

#### 6. It is important to understand that the [Attributes](#attributelist) are NOT self updating (except for the [Static](#staticattributes) ones which are created when the Sensor Object is initialized).
#### 7. To guarantee up-to-date data, the appropriate [Methods](#methodlist) must be called to update the [Attribute](#attributelist) to be read for each Sensor Object BEFORE the Sensor Object's state is read.

<a id='exampletemperature'></a>
---

# Part 10: Example Temperature
#### 1. Call the updateTemperature() [Method](#methodlist)

In [12]:
motor_inboard.updateTemperature()

#### 2. Read the [Attribute](#attributelist) that was Updated

In [13]:
motor_inboard.temperature

{'celcius': 17.7, 'fahrenheit': 63.86}

<a id='examplevibration'></a>
---

# Part 11: Example Impact
#### 1. Call the setImpactDefaults() [Method](#methodlist)

In [14]:
motor_inboard.impact

{'machine_speed_rpm': 960,
 'alert_threshold_gpk': 5.0,
 'danger_threshold_gpk': 15.0}

#### 2. You Must Wait For At Least One Impact Window to Complete Before Updating Impact
#### 3. Call the updateImpact() [Method](#methodlist)
#### 4. View the change to your Impact Attribute

In [ ]:
motor_inboard._alarmHighest()

In [16]:
time.sleep(0)
motor_inboard.updateImpact()
motor_inboard.impact

{'machine_speed_rpm': 960,
 'alert_threshold_gpk': 5.0,
 'danger_threshold_gpk': 15.0,
 'alert_counts': 0.0,
 'danger_counts': 0.0,
 'severity': 0.0,
 'temperature': 0.0}

In [ ]:
# Set the RPM of your Machine
motor_inboard._alarmStatus()

In [ ]:
motor_inboard._alarmSettings()

#### 2. Read the [Attribute](#attributelist) that was Updated

<a id='easysnapshot'></a>
---

## Part 12: Example Detailed Snapshot  
#### 1. Let's call the snapshot [Attribute](#attributelist) for our example sensor.

In [ ]:
motor_inboard.snapshot

#### 2. Notice how the snapshot object starts out empty within the sensor object.
#### 3. This is because we have not yet called the Methods which update this Attribute.
#### 4. Lets do that now using the easySnapshot() [Method](#methodlist).

# Setup Snapshot Parameters

In [ ]:
motor_inboard.setSnapshotTime(2000)
motor_inboard.snapshot

In [ ]:
motor_inboard.setSnapshotSamples(16384)
motor_inboard.snapshot

In [ ]:
motor_inboard.setSnapshotAccel(5)
motor_inboard.snapshot.get("internal_accelerometer")

# Check Snapshot Engine is Idle before Triggering Snapshot

In [ ]:
while(motor_inboard.snapshot.get("capture_engine_status")!="Idle"):
    motor_inboard.updateSnapshotEngineStatus()
    print(motor_inboard.snapshot.get("capture_engine_status"))
    time.sleep(0.5)
motor_inboard.triggerSnapshot()

# Check Snapshot Engine is Complete before Snapshot Collect

In [ ]:
while(motor_inboard.snapshot.get("capture_engine_status")!= "Complete"):
    motor_inboard.updateSnapshotEngineStatus()
    print(motor_inboard.snapshot.get("capture_engine_status"))
    time.sleep(0.5)
motor_inboard.collectSnapshot()

# View Gathered Snapshot Object

In [ ]:
# motor_inboard.snapshot.get("triaxial_RAW_samples")

The triaxial data set will likely have trailing zeroes because the total samples rarely fit cleanly into our 122 modbus register read.
So we need to slice the triaxial set up into individual axis using samples per axis which will ignore the padded zeroes at the end.

In [ ]:
motor_inboard.snapshot.get("axis_1_RAW")
print(min(motor_inboard.snapshot.get("axis_1_RAW")))
print(max(motor_inboard.snapshot.get("axis_1_RAW")))
print(np.size(motor_inboard.snapshot.get("axis_1_RAW")))

In [ ]:
motor_inboard.snapshot.get("axis_2_RAW")
print(min(motor_inboard.snapshot.get("axis_2_RAW")))
print(max(motor_inboard.snapshot.get("axis_2_RAW")))
print(np.size(motor_inboard.snapshot.get("axis_2_RAW")))

In [ ]:
motor_inboard.snapshot.get("axis_3_RAW")
print(min(motor_inboard.snapshot.get("axis_3_RAW")))
print(max(motor_inboard.snapshot.get("axis_3_RAW")))
print(np.size(motor_inboard.snapshot.get("axis_3_RAW")))

In [ ]:
print(np.size(motor_inboard.snapshot.get("time_array_ms")))
print(max(motor_inboard.snapshot.get("time_array_ms")))

Now that the minimal amount of data has been generated for the sensor, we can create an excel sheet with the data and save it and export it to the cloud (and process the FFT not using the precious resources of our embedded PC).

In [ ]:
motor_inboard.createSnapshotXLSX()

### [Back to Top](#top)  